In [1]:
import sys, os, itertools
import re, nltk
from nltk import pos_tag
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import sklearn, gensim
from sklearn.decomposition import PCA
from gensim.corpora import Dictionary
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
os.chdir(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/data/')
from utils import preprocess_for_bow
from models.lda import LDAwrappers
from models.hdp import HDPwrapper
from models.gsdmm import MovieGroupProcessWrapper
from models.lftm import LFTMwrapper
from models.bertopic import BERTopicWrapper
from sklearn.feature_extraction.text import CountVectorizer
from scipy.cluster import hierarchy as sch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
from bertopic.representation import TextGeneration

generator = pipeline('text2text-generation', model='google/flan-t5-base')
representation_model = TextGeneration(generator, "I have a topic described by the following keywords: [KEYWORDS]. Based on the previous keywords, what is this topic about?")

In [2]:
from bertopic.representation import MaximalMarginalRelevance, KeyBERTInspired
keybert = KeyBERTInspired()
mmr = MaximalMarginalRelevance(diversity=0.3)
chained_representation = [keybert, mmr]

In [3]:
data1 = preprocess_for_bow('data.csv', preprocessing=False)
data2 = preprocess_for_bow('claimreviews.csv', preprocessing=False)

## dataset 1

In [4]:
bertopic1=BERTopicWrapper(data1['text'], data1['ids'])
bertopic1.get_topic_info()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12649,-1_covid_is_of_the,"[covid, is, of, the, they, for, are, in, by, 19]",[They think it's a Globalist conspiracy to cre...
1,0,580,0_masks_mask_face_wearing,"[masks, mask, face, wearing, wear, ineffective...",[Children should not wear masks or be vaccinat...
2,1,551,1_pope_church_catholic_bishops,"[pope, church, catholic, bishops, cardinal, fr...","[This is how one explains that, as members of ..."
3,2,550,2_tweet_contribute_menu_helpful,"[tweet, contribute, menu, helpful, might, misl...",[If you want to add helpful context to any Twe...
4,3,298,3_ballots_mail_ballot_votes,"[ballots, mail, ballot, votes, voter, fraud, v...",[Wisconsin has more ballots cast than register...
...,...,...,...,...,...
696,695,6,695_emissions_coal_nz_morrison,"[emissions, coal, nz, morrison, carbon, cent, ...",[NZ National leader Judith Collins claims coal...
697,696,6,696_nanggala_kri_402_submarine,"[nanggala, kri, 402, submarine, sunken, indone...",[Video shows Indonesia has salvaged wreckage o...
698,697,6,697_lincoln_destroying_abraham_danger,"[lincoln, destroying, abraham, danger, falter,...","[Abraham Lincoln once said, """"You cannot help ..."
699,698,6,698_print_page_flobar_paper,"[print, page, flobar, paper, pdf, 1950s, deter...",[A mysterious piece of paper discovered last w...


In [5]:
print(bertopic1.get_topic_info().iloc[:10,[0,1,3]])
for i in range(10):
    print(bertopic1.get_topic_info().iloc[i,3])

   Topic  Count                                     Representation
0     -1  12649   [covid, is, of, the, they, for, are, in, by, 19]
1      0    580  [masks, mask, face, wearing, wear, ineffective...
2      1    551  [pope, church, catholic, bishops, cardinal, fr...
3      2    550  [tweet, contribute, menu, helpful, might, misl...
4      3    298  [ballots, mail, ballot, votes, voter, fraud, v...
5      4    245  [gates, bill, microsoft, melinda, depopulation...
6      5    237  [israel, israeli, gaza, palestinian, palestini...
7      6    192  [gun, guns, amendment, ar, weapons, nra, backg...
8      7    185  [duterte, philippine, marcos, rodrigo, robredo...
9      8    159  [woof, true, validation, unchanged, sums, wild...
['covid', 'is', 'of', 'the', 'they', 'for', 'are', 'in', 'by', '19']
['masks', 'mask', 'face', 'wearing', 'wear', 'ineffective', 'vaccinated', 'spread', 'work', 'protect']
['pope', 'church', 'catholic', 'bishops', 'cardinal', 'francis', 'vatican', 'archbishop', '

FLAN generated labels

In [6]:
bertopic1.update_topics(data1['text'], representation_model=representation_model)
print(bertopic1.get_topic_info().iloc[:10,[0,1,3]])
for i in range(10):
    print(bertopic1.get_topic_info().iloc[i,3])

   Topic  Count                    Representation
0     -1  12649         [World, , , , , , , , , ]
1      0    580  [Science/Tech, , , , , , , , , ]
2      1    551      [religion, , , , , , , , , ]
3      2    550     [birdwatch, , , , , , , , , ]
4      3    298      [election, , , , , , , , , ]
5      4    245  [Science/Tech, , , , , , , , , ]
6      5    237      [conflict, , , , , , , , , ]
7      6    192   [gun control, , , , , , , , , ]
8      7    185         [World, , , , , , , , , ]
9      8    159  [Science/Tech, , , , , , , , , ]
['World', '', '', '', '', '', '', '', '', '']
['Science/Tech', '', '', '', '', '', '', '', '', '']
['religion', '', '', '', '', '', '', '', '', '']
['birdwatch', '', '', '', '', '', '', '', '', '']
['election', '', '', '', '', '', '', '', '', '']
['Science/Tech', '', '', '', '', '', '', '', '', '']
['conflict', '', '', '', '', '', '', '', '', '']
['gun control', '', '', '', '', '', '', '', '', '']
['World', '', '', '', '', '', '', '', '', '']
['S

custom representation model

In [7]:
bertopic1.update_topics(data1['text'], representation_model=chained_representation)
print(bertopic1.get_topic_info().iloc[:10,[0,1,3]])
for i in range(10):
    print(bertopic1.get_topic_info().iloc[i,3])

   Topic  Count                                     Representation
0     -1  12649  [pandemic, covid, coronavirus, vaccine, vaccin...
1      0    580  [masks, mask, masking, maskless, prevention, p...
2      1    551  [pope, catholic, catholics, vatican, bishops, ...
3      2    550  [birdwatch, tweet, contributions, contribute, ...
4      3    298  [ballots, voter, ballot, voters, elections, re...
5      4    245  [vaccine, microsoft, melinda, vaccines, virus,...
6      5    237  [gaza, palestinian, palestine, palestinians, h...
7      6    192  [firearm, firearms, handgun, guns, gun, rifle,...
8      7    185  [marcos, philippine, philippines, aquino, dute...
9      8    159  [woof, truthout, true, field, statement, know,...
['pandemic', 'covid', 'coronavirus', 'vaccine', 'vaccines', 'trump', 'donald', 'president', 'virus', 'news']
['masks', 'mask', 'masking', 'maskless', 'prevention', 'prevent', 'covid', 'vaccinated', 'face', 'immune']
['pope', 'catholic', 'catholics', 'vatican', 'b

representation model + vectorizer

In [8]:
bertopic1.update_topics(data1['text'], vectorizer_model=CountVectorizer(strip_accents= None, stop_words= "english",
                        ngram_range= (1,2)), representation_model=chained_representation)
print(bertopic1.get_topic_info().iloc[:10,[0,1,3]])
for i in range(10):
    print(bertopic1.get_topic_info().iloc[i,3])

   Topic  Count                                     Representation
0     -1  12649  [pandemic, covid 19, trump, donald trump, pres...
1      0    580  [wear masks, face masks, masks prevent, wear m...
2      1    551  [pope, pope francis, catholic bishops, letter ...
3      2    550  [contribute birdwatch, contributions birdwatch...
4      3    298  [mail ballots, mail ballot, mail voting, ballo...
5      4    245  [melinda gates, founder gates, gates funded, g...
6      5    237  [hamas video, israel video, palestine video, i...
7      6    192  [gun control, gun confiscation, second amendme...
8      7    185  [bongbong marcos, marcos, ferdinand marcos, ma...
9      8    159  [woof woof, woof, phone field, field validatio...
['pandemic', 'covid 19', 'trump', 'donald trump', 'president donald', 'covid', 'coronavirus', 'covid19', 'donald', '2020']
['wear masks', 'face masks', 'masks prevent', 'wear mask', 'wearing masks', 'face mask', 'masks protect', 'wearing mask', 'masks ineffective

FLAN generated labels

In [9]:
bertopic1.update_topics(data1['text'], representation_model=representation_model)
print(bertopic1.get_topic_info().iloc[:10,[0,1,3]])
for i in range(10):
    print(bertopic1.get_topic_info().iloc[i,3])

   Topic  Count                    Representation
0     -1  12649         [World, , , , , , , , , ]
1      0    580  [Science/Tech, , , , , , , , , ]
2      1    551      [religion, , , , , , , , , ]
3      2    550     [birdwatch, , , , , , , , , ]
4      3    298      [election, , , , , , , , , ]
5      4    245  [Science/Tech, , , , , , , , , ]
6      5    237      [conflict, , , , , , , , , ]
7      6    192   [gun control, , , , , , , , , ]
8      7    185         [World, , , , , , , , , ]
9      8    159  [Science/Tech, , , , , , , , , ]
['World', '', '', '', '', '', '', '', '', '']
['Science/Tech', '', '', '', '', '', '', '', '', '']
['religion', '', '', '', '', '', '', '', '', '']
['birdwatch', '', '', '', '', '', '', '', '', '']
['election', '', '', '', '', '', '', '', '', '']
['Science/Tech', '', '', '', '', '', '', '', '', '']
['conflict', '', '', '', '', '', '', '', '', '']
['gun control', '', '', '', '', '', '', '', '', '']
['World', '', '', '', '', '', '', '', '', '']
['S

saves indexes of label -1 and refit bertopic

In [10]:
text_main_topic = []
for idx in np.where(np.array(bertopic1.topics_)==-1)[0]:
    text_main_topic.append(data1['text'][idx])

In [11]:
bertopic11=BERTopicWrapper(text_main_topic, None, representation_model=chained_representation)
bertopic11.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,12391,0_covid_pandemic_vaccine_coronavirus,"[covid, pandemic, vaccine, coronavirus, virus,...",[They think it's a Globalist conspiracy to cre...
1,1,204,1_estado_una_como_regalando,"[estado, una, como, regalando, es, por, protes...",[Ya saltó un p^%*~o a decir que “hay que aboli...
2,2,47,2_earthquake_earthquakes_quake_san,"[earthquake, earthquakes, quake, san, californ...",[4.3 earthquake hits off NW California - WARNI...
3,3,7,3_bahrain_bodyguard_dubai_arabia,"[bahrain, bodyguard, dubai, arabia, saudi, kin...",[King of Bahrain arrives in Dubai with his rob...


In [12]:
print(bertopic11.get_topic_info().iloc[:10,[0,1,3]])
for i in range(10):
    print(bertopic11.get_topic_info().iloc[i,3])

   Topic  Count                                     Representation
0      0  12391  [covid, pandemic, vaccine, coronavirus, virus,...
1      1    204  [estado, una, como, regalando, es, por, protes...
2      2     47  [earthquake, earthquakes, quake, san, californ...
3      3      7  [bahrain, bodyguard, dubai, arabia, saudi, kin...
['covid', 'pandemic', 'vaccine', 'coronavirus', 'virus', '2020', 'trump', 'donald', '2021', 'us']
['estado', 'una', 'como', 'regalando', 'es', 'por', 'protesta', 'autêntico', 'que', 'venezuela']
['earthquake', 'earthquakes', 'quake', 'san', 'california', 'angeles', 'francisco', 'ca', 'los', 'central']
['bahrain', 'bodyguard', 'dubai', 'arabia', 'saudi', 'king', 'robot', 'arrives', 'his', 'arriving']


IndexError: index 4 is out of bounds for axis 0 with size 4

refit model on topic 0

In [13]:
text_main_topic = []
for idx in np.where(np.array(bertopic11.topics_)==0)[0]:
    text_main_topic.append(data1['text'][idx])

In [14]:
bertopic12=BERTopicWrapper(text_main_topic, None, representation_model=chained_representation)
bertopic12.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4982,-1_coronavirus_covid_pandemic_vaccine,"[coronavirus, covid, pandemic, vaccine, virus,...",[. . The fake news media and corrupt deep stat...
1,0,534,0_birdwatch_tweet_contributions_contribute,"[birdwatch, tweet, contributions, contribute, ...",[If you want to add helpful context to any Twe...
2,1,310,1_masks_mask_masking_vaccinated,"[masks, mask, masking, vaccinated, covid, immu...","[Get vaccinated or wear a mask until you do., ..."
3,2,199,2_biden_bidenlied_presidential_trump,"[biden, bidenlied, presidential, trump, votes,...",[Don't let them make you forget - we all liter...
4,3,157,3_caeser_pounce_locallytweeted_mattrichardsga_...,"[caeser_pounce, locallytweeted, mattrichardsga...",[@caeser_pounce @MattyThrice @neontaster If @M...
...,...,...,...,...,...
253,252,6,252_nra_guns_shooting_inciting,"[nra, guns, shooting, inciting, massacre, righ...",[AP has deleted a tweet that incorrectly said ...
254,253,6,253_coronavirus_china_chinese_virus,"[coronavirus, china, chinese, virus, corona, s...",[Chinese Intelligence Officer termed coronavir...
255,254,6,254_murdered_riots_capitol_policeman,"[murdered, riots, capitol, policeman, killed, ...",[Stop living in the past by focusing on the Ca...
256,255,6,255_capitol_threat_misinformation_violent,"[capitol, threat, misinformation, violent, gop...",[While there is some talk of troops going home...


In [15]:
print(bertopic12.get_topic_info().iloc[:10,[0,1,3]])
for i in range(10):
    print(bertopic12.get_topic_info().iloc[i,3])

   Topic  Count                                     Representation
0     -1   4982  [coronavirus, covid, pandemic, vaccine, virus,...
1      0    534  [birdwatch, tweet, contributions, contribute, ...
2      1    310  [masks, mask, masking, vaccinated, covid, immu...
3      2    199  [biden, bidenlied, presidential, trump, votes,...
4      3    157  [caeser_pounce, locallytweeted, mattrichardsga...
5      4    127  [gaza, gazaunderattack, gazaunderattak, palest...
6      5    118  [schools, classrooms, nycschools, students, sc...
7      6    116  [prayer, covid, pray, spiritual, spiritually, ...
8      7    110  [vaccine, vaccines, vaccination, covid, vaccin...
9      8    100  [melania, kardashian, trump, hillary, donald, ...
['coronavirus', 'covid', 'pandemic', 'vaccine', 'virus', 'news', 'trump', 'china', 'us', 'me']
['birdwatch', 'tweet', 'contributions', 'contribute', 'gretathunbergexposed', 'context', 'link', 'helpful', 'on', 'to']
['masks', 'mask', 'masking', 'vaccinated', 'covi

## dataset2

In [16]:
bertopic2=BERTopicWrapper(data2['text'], data2['ids'])
bertopic2.get_topic_info()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

,Topic,Count,Name,Representation,Representative_Docs
0,-1,6459,-1_the_to_and_was,"[the, to, and, was, of, that, on, in, is, with]",[A compilation of three different videos is be...
1,0,336,0_duterte_rappler_philippine_marcos,"[duterte, rappler, philippine, marcos, philipp...","[SUMMARYThis is AI generated summarization, wh..."
2,1,195,1_masks_mask_face_wearing,"[masks, mask, face, wearing, wear, cloth, surg...",[Misleading Graphics About Dangers Of Long-ter...
3,2,169,2_incident_police_beaten_communal,"[incident, police, beaten, communal, video, ac...",[Video Of Cricket Bat Assault On Temple Priest...
4,3,144,3_पर_यह_एक_इस,"[पर, यह, एक, इस, यरल, तस, हम, और, रह, सकत]",[Fact Check: कानपुर के इस वीडियो में दिख रहा व...
...,...,...,...,...,...
527,526,6,526_springs_nps_park_paige,"[springs, nps, park, paige, hot, yellowstone, ...",[Were law enforcement officers investigating s...
528,527,6,527_teacher_fictional_story_fahim,"[teacher, fictional, story, fahim, class, stud...",[Several social media users shared a viral ima...
529,528,6,528_ہے_کے_میں_کی,"[ہے, کے, میں, کی, کو, کا, پر, کہ, نے, ویڈیو]",[فیکٹ چیک: ویڈیو کا وارانسی سے کوئی تعلق نہیں ...
530,529,6,529_ireland_tomlinson_deaths_northern,"[ireland, tomlinson, deaths, northern, death, ...",[“Fatalities are barely any higher”In an artic...


In [17]:
print(bertopic2.get_topic_info().iloc[:10,[0,1,3]])
for i in range(10):
    print(bertopic2.get_topic_info().iloc[i,3])

   Topic  Count                                     Representation
0     -1   6459    [the, to, and, was, of, that, on, in, is, with]
1      0    336  [duterte, rappler, philippine, marcos, philipp...
2      1    195  [masks, mask, face, wearing, wear, cloth, surg...
3      2    169  [incident, police, beaten, communal, video, ac...
4      3    144         [पर, यह, एक, इस, यरल, तस, हम, और, रह, सकत]
5      4    121  [trump, administration, president, pandemic, t...
6      5    117  [sri, lanka, lankan, sinhala, rajapaksa, misle...
7      6    117       [em, da, de, que, um, uma, não, os, na, foi]
8      7    114  [soldiers, chinese, galwan, indian, army, lada...
9      8    101  [gates, bill, foundation, melinda, polio, micr...
['the', 'to', 'and', 'was', 'of', 'that', 'on', 'in', 'is', 'with']
['duterte', 'rappler', 'philippine', 'marcos', 'philippines', 'robredo', 'aquino', 'manila', 'ang', 'comments']
['masks', 'mask', 'face', 'wearing', 'wear', 'cloth', 'surgical', 'coverings', 'dro

In [18]:
bertopic2.update_topics(data2['text'], representation_model=representation_model)
print(bertopic2.get_topic_info().iloc[:10,[0,1,3]])
for i in range(10):
    print(bertopic2.get_topic_info().iloc[i,3])

   Topic  Count                           Representation
0     -1   6459         [Science/Tech, , , , , , , , , ]
1      0    336       [disinformation, , , , , , , , , ]
2      1    195         [Science/Tech, , , , , , , , , ]
3      2    169  [video clip savarkar, , , , , , , , , ]
4      3    144                     [, , , , , , , , , ]
5      4    121               [health, , , , , , , , , ]
6      5    117                [World, , , , , , , , , ]
7      6    117               [ese da, , , , , , , , , ]
8      7    114                [World, , , , , , , , , ]
9      8    101         [Science/Tech, , , , , , , , , ]
['Science/Tech', '', '', '', '', '', '', '', '', '']
['disinformation', '', '', '', '', '', '', '', '', '']
['Science/Tech', '', '', '', '', '', '', '', '', '']
['video clip savarkar', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '']
['health', '', '', '', '', '', '', '', '', '']
['World', '', '', '', '', '', '', '', '', '']
['ese da', '', '',

In [19]:
bertopic2.update_topics(data2['text'], vectorizer_model=CountVectorizer(strip_accents= None, stop_words= "english",
                        ngram_range= (1,2)), representation_model=chained_representation)
print(bertopic2.get_topic_info().iloc[:10,[0,1,3]])
for i in range(10):
    print(bertopic2.get_topic_info().iloc[i,3])

   Topic  Count                                     Representation
0     -1   6459  [covid 19, coronavirus, covid, viral, vaccine,...
1      0    336  [rodrigo duterte, marcos, president rodrigo, p...
2      1    195  [wear masks, wearing masks, wear mask, surgica...
3      2    169  [viral video, policeman, policemen, violence, ...
4      3    144  [यरल रह, रह, हम, हम बत, सच चन, असल, रम, पर हम,...
5      4    121  [coronavirus hoax, covid 19, pandemic, 19 coro...
6      5    117  [sri lankan, lankan government, lanka, sri lan...
7      6    117  [este conteúdo, conteúdo, verificação, avaliaç...
8      7    114  [chinese soldiers, chinese troops, chinese arm...
9      8    101  [polio vaccine, vaccine, vaccines, anti vaccin...
['covid 19', 'coronavirus', 'covid', 'viral', 'vaccine', 'virus', 'news', 'incident', 'health', 'twitter']
['rodrigo duterte', 'marcos', 'president rodrigo', 'pacquiao', 'duterte', 'aquino', 'philippine', 'rodrigo', 'filipino', 'manila']
['wear masks', 'wearing m

In [20]:
bertopic2.update_topics(data2['text'], representation_model=representation_model)
print(bertopic2.get_topic_info().iloc[:10,[0,1,3]])
for i in range(10):
    print(bertopic2.get_topic_info().iloc[i,3])

   Topic  Count                           Representation
0     -1   6459         [Science/Tech, , , , , , , , , ]
1      0    336       [disinformation, , , , , , , , , ]
2      1    195         [Science/Tech, , , , , , , , , ]
3      2    169  [video clip savarkar, , , , , , , , , ]
4      3    144                     [, , , , , , , , , ]
5      4    121               [health, , , , , , , , , ]
6      5    117                [World, , , , , , , , , ]
7      6    117               [ese da, , , , , , , , , ]
8      7    114                [World, , , , , , , , , ]
9      8    101         [Science/Tech, , , , , , , , , ]
['Science/Tech', '', '', '', '', '', '', '', '', '']
['disinformation', '', '', '', '', '', '', '', '', '']
['Science/Tech', '', '', '', '', '', '', '', '', '']
['video clip savarkar', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '']
['health', '', '', '', '', '', '', '', '', '']
['World', '', '', '', '', '', '', '', '', '']
['ese da', '', '',

In [21]:
text2_main_topic = []
for idx in np.where(np.array(bertopic2.topics_)==-1)[0]:
    text2_main_topic.append(data1['text'][idx])

In [22]:
bertopic21=BERTopicWrapper(text2_main_topic, None, representation_model=chained_representation)
bertopic21.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2862,-1_twitter_news_trump_donald,"[twitter, news, trump, donald, of, he, to, me,...",[If you want to add helpful context to any Twe...
1,0,147,0_masks_mask_vaccinated_immune,"[masks, mask, vaccinated, immune, covid, covid...",[The science is clear: If you are vaccinated a...
2,1,127,1_biden_bidenlied_presidential_president,"[biden, bidenlied, presidential, president, im...",[Joe Biden is not my President. Legally not my...
3,2,106,2_es_una_como_une,"[es, una, como, une, por, un, protestas, el, e...",[En @UInvestiga obtuvimos una copia del expedi...
4,3,95,3_email_lodged_jenfl23_posting,"[email, lodged, jenfl23, posting, ll, miko_pik...",[@AstroAlysa Defamation is a crime. The false ...
...,...,...,...,...,...
175,174,6,174_holiday_juneteenth_celebrated_christmas,"[holiday, juneteenth, celebrated, christmas, 3...",[Happy Juneteenth! The first U.S. holiday that...
176,175,6,175_cyclone_typhoon_tornado_bengal,"[cyclone, typhoon, tornado, bengal, kerala, ba...",[This video shows damage caused by cyclone Amp...
177,176,6,176_presidentsday_president_presidents_presidento,"[presidentsday, president, presidents, preside...","[🇺🇸 Happy Presidents Day Donald J Trump 🇺🇸, Ha..."
178,177,6,177_blm_riots_protests_terrorist,"[blm, riots, protests, terrorist, riot, violen...",[DC is completely dead tonight. People stayed ...


In [23]:
print(bertopic21.get_topic_info().iloc[:10,[0,1,3]])
for i in range(10):
    print(bertopic21.get_topic_info().iloc[i,3])

   Topic  Count                                     Representation
0     -1   2862  [twitter, news, trump, donald, of, he, to, me,...
1      0    147  [masks, mask, vaccinated, immune, covid, covid...
2      1    127  [biden, bidenlied, presidential, president, im...
3      2    106  [es, una, como, une, por, un, protestas, el, e...
4      3     95  [email, lodged, jenfl23, posting, ll, miko_pik...
5      4     94  [coronavirus, coronavaccin, outbreak, virus, e...
6      5     92  [vaccine, vaccines, vaccination, vaccinated, v...
7      6     75  [statue, moscow, soviet, president, donald, in...
8      7     67  [schools, teachers, vaccinated, nycschools, st...
9      8     54  [nigeria, nigerian, nigerians, africa, ghana, ...
['twitter', 'news', 'trump', 'donald', 'of', 'he', 'to', 'me', 'what', 'said']
['masks', 'mask', 'vaccinated', 'immune', 'covid', 'covid19', 'pandemic', 'infections', 'protect', 'wear']
['biden', 'bidenlied', 'presidential', 'president', 'impeachbiden', 'trump', 

In [24]:
bertopic21.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2862,-1_twitter_news_trump_donald,"[twitter, news, trump, donald, of, he, to, me,...",[If you want to add helpful context to any Twe...
1,0,147,0_masks_mask_vaccinated_immune,"[masks, mask, vaccinated, immune, covid, covid...",[The science is clear: If you are vaccinated a...
2,1,127,1_biden_bidenlied_presidential_president,"[biden, bidenlied, presidential, president, im...",[Joe Biden is not my President. Legally not my...
3,2,106,2_es_una_como_une,"[es, una, como, une, por, un, protestas, el, e...",[En @UInvestiga obtuvimos una copia del expedi...
4,3,95,3_email_lodged_jenfl23_posting,"[email, lodged, jenfl23, posting, ll, miko_pik...",[@AstroAlysa Defamation is a crime. The false ...
...,...,...,...,...,...
175,174,6,174_holiday_juneteenth_celebrated_christmas,"[holiday, juneteenth, celebrated, christmas, 3...",[Happy Juneteenth! The first U.S. holiday that...
176,175,6,175_cyclone_typhoon_tornado_bengal,"[cyclone, typhoon, tornado, bengal, kerala, ba...",[This video shows damage caused by cyclone Amp...
177,176,6,176_presidentsday_president_presidents_presidento,"[presidentsday, president, presidents, preside...","[🇺🇸 Happy Presidents Day Donald J Trump 🇺🇸, Ha..."
178,177,6,177_blm_riots_protests_terrorist,"[blm, riots, protests, terrorist, riot, violen...",[DC is completely dead tonight. People stayed ...
